# 패키지 불러오기

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings( 'ignore' )

In [3]:
pd.options.display.float_format = '{:.5f}'.format

In [1]:
medium = pd.read_csv("data/Medium_Theater_estimate.csv")

## 월별합이 0인애들 제거

In [30]:
temp = medium.groupby(["공연시설코드", "년도_월"]).sum().reset_index()

In [31]:
temp = temp[temp["예매/취소금액"]==0][["공연시설코드", "년도_월"]]

In [33]:
code = temp["공연시설코드"].tolist()
month = temp["년도_월"].tolist()

In [34]:
idx = []
for i,j in zip(code, month):
    num = medium[(medium["공연시설코드"]==i)&(medium["년도_월"]==j)].index
    idx.append(num)

In [36]:
idx = [x for idx in idx for x in idx.tolist()]

In [38]:
medium = medium.drop(idx, axis=0)

## 좌석수평균 X 평균좌석당금액

In [49]:
# 19년도 데이터가 1개인 데이터는 그 가격 그대로 예상매출액으로 사용.
code = medium["공연시설코드"].unique()
except_lst = []

for i in code:
    temp = medium[medium["공연시설코드"]==i]
    num = len(temp)
    if num == 1:
        except_lst.append(i)
except_lst

['FC222040',
 'FC222089',
 'FC222272',
 'FC222466',
 'FC222564',
 'FC222622',
 'FC222628',
 'FC222741',
 'FC222964',
 'FC222982',
 'FC223493',
 'FC224381',
 'FC224496']

In [50]:
def MakeExpectedSales(code):
    if code in except_lst:
        temp = medium[medium["공연시설코드"]==code]
        res = temp["예매/취소금액"].values[0]
    else:
        temp = medium[medium["공연시설코드"]==code]
        temp["좌석당가격"] = temp["예매/취소금액"] / temp["좌석수"]
        gb = temp.groupby("년도_월").sum()
        seat_mean = temp["좌석수"].mean()
        seat_price = gb["좌석당가격"].mean()
        res = seat_mean * seat_price
    return res

In [51]:
code = medium["공연시설코드"].unique()
expected = []

for i in code:
    res = MakeExpectedSales(i)
    expected.append(res)

In [52]:
res_df = pd.DataFrame({"공연시설코드":code, "좌석당금액":expected})
res_df

,공연시설코드,좌석당금액
0,FC222007,17877850.50000
1,FC222040,4000000.00000
2,FC222048,31831533.00000
3,FC222049,2469018.28571
4,FC222054,10738874.42857
...,...,...
133,FC223909,109597074.27051
134,FC223941,16358687.50000
135,FC224381,1404000.00000
136,FC224454,1794000.00000


In [53]:
# res_df.to_csv("중극장_예상매출액_평균좌석당금액이용.csv", index=False)